In [140]:
import os
print(os.getcwd())

import platform
platform.python_version()

import pandas as pd

/home/ubuntu


In [141]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [3]:
USER = !whoami
USER = USER[0]
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

In [7]:
!export HADOOP_HOME=/usr/local/hadoop
!export HADOOP_OPTS="-Dhadoop.tmp.dir=/user/ubuntu/tmp"
#!sudo ln -s /usr/lib/hadoop-mapreduce/hadoop-streaming.jar /usr/lib/hadoop

In [98]:
#!head -n 100 /data_data/History_file_1000000_trials_2_sides_Feb_28_2020_0719 >  /data_data/History_100.csv
#!hdfs dfs -put /data_data/History_100.csv {OUTPUT_PATH_BASE}/data

In [4]:
#!ls -lh /data_data/reinforcement_learning/results/

In [1]:
!mv /data_data/reinforcement_learning/results/history_file_2000000_trials_2_sides_May_05_2020_0939 /data_data/reinforcement_learning/results/history_file_2M_test.csv

In [9]:
#!head -n 5 /data_data/reinforcement_learning/results/history_file_2M_test.csv

In [10]:
%%writefile transitionMatrix.py

from __future__ import division
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict, Counter
import re
import ast


class MRtransitionMatrix(MRJob):
    
    def steps(self):
        return [MRStep(
                    mapper=self.mapper,
                   reducer=self.reducer)]
            
    def mapper(self, _, lines_):
        lines = lines_.split("\n")

        for line_ in lines:
            try:
                cycle,trans = line_.replace("\"","'").split("\t")
                transitions = ast.literal_eval(trans)
                for item in transitions:
                    move_count = int(item[0])
                    if move_count % 2 == 0:
                        yield int(cycle), item
            except:
                pass

                
        
        
    def reducer(self, history, values):
        episodes = []
        
        initial_state = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]
        
        piece_value_dict = {"0":0, "1":5,"2":3,"3":3,"4":9,"5":20,"6":3,"7":3,"8":5,"9":1,"10":1,"11":1,"12":1,"13":1,"14":1,"15":1,"16":1,"-16":-1,"-15":-1,"-14":-1,"-13":-1,"-12":-1,"-11":-1,"-10":-1,"-9":-1,"-8":-5,"-7":-3,"-6":-3,"-5":-9,"-4":-20,"-3":-3,"-2":-3,"-1":-5}
        
        history_ = []
        
        for value in values:
            #num_turns += 1
            try:

                episodes.append(value)   
                
            except:
                pass
            
        #yield history, episode          
        
        num_states = len(episodes) 
        
        for num in range(num_states):
            state = ast.literal_eval(episodes[num][1])
            
            try:
                state_prime = ast.literal_eval(episodes[num+1][1])
            except:
                state_prime = ast.literal_eval(episodes[num][1])
                
            move        = ast.literal_eval(episodes[num][0])
            value       = sum([int(piece_value_dict[str(x)]) for x in state])
            value_prime = sum([int(piece_value_dict[str(x)]) for x in state_prime])
            action      = episodes[num][3]
            reward      = value_prime - value
            Value       = 0
            
            history_.append([history,[move, state, action, state_prime, value, value_prime, reward, Value]])
            
                            
        rewards = []
                            
        for histry in history_:
            rewards.append(histry[1][6])
        
        num_values = len(rewards)

        gamma = 0.5

        counter = 0

        Values = []

        for reward in rewards[::-1]:
            if counter == 0:
                Values.append(round(reward,6))
            else:
                Values.append(round(reward + gamma*Values[counter-1],6))

            counter += 1

        Values = Values[::-1]

        for val in range(num_values):
            
            #yield str(type(Values[val])), str(type(history_[val][1][7]))
            history_[val][1][7] = Values[val]
            
        for hist in history_:
            #yield history, ({"state_action": (hist[1][1],hist[1][2]),"state_prime": hist[1][3], "value": hist[1][7]})
            #yield (hist[1][1], hist[1][2]), hist[1][3]
            yield None, {history: {"state": hist[1][1],"action":hist[1][2],"state_prime": hist[1][3], "reward":reward, "Value": hist[1][7]}}
        
if __name__ == '__main__':
    MRtransitionMatrix.run() 

Overwriting transitionMatrix.py


In [11]:
!whereis source

source:


In [12]:
#!env

In [13]:

#!hdfs dfs -ls /user/ubuntu/data/history_file_500K_test.csv

In [14]:
!echo $PATH

/home/ubuntu/.local/bin:/home/ubuntu/anaconda3/bin:/home/ubuntu/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/hadoop/hadoop-2.8.0/sbin:/home/hadoop/hadoop-2.8.0/bin


In [15]:
#!hdfs dfs -rm /user/ubuntu/data/history_file_500K_test.csv

!rm -rf /data_data/reinforcement_learning/results/sarsa

!rm -rf /data_data/reinforcement_learning/results/history_sarsa_2M_test

!python /home/ubuntu/transitionMatrix.py /data_data/reinforcement_learning/results/history_file_2M_test.csv -r local --output-dir=/data_data/reinforcement_learning/results/sarsa 

#!hdfs dfs -put /data_data/reinforcement_learning/results/history_file_500K_test.csv /user/ubuntu/data/

!cat /data_data/reinforcement_learning/results/sarsa/part-* >> /data_data/reinforcement_learning/results/history_sarsa_2M_test.tsv

!rm -rf /data_data/reinforcement_learning/results/sarsa

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/transitionMatrix.ubuntu.20200506.131254.254523
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/sarsa
Removing temp directory /tmp/transitionMatrix.ubuntu.20200506.131254.254523...


In [17]:
!tail -n 5 /data_data/reinforcement_learning/results/history_sarsa_2M_test.tsv

null	{"999999":{"state":[1,2,3,4,0,5,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,7,8,9,0,0,0,0,0,10,11,0,0,0,0,0,0,0,0,0,12,0,0,13,14,0,0,0,15,16,-16,0,-15,0,-14],"action":"[Rook,(1,0),(3,7),(4,7)]","state_prime":[1,2,3,4,0,5,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,8,0,0,0,0,0,9,10,0,0,11,0,0,0,0,0,0,0,0,0,12,13,0,0,0,14,15,16,0,-16,0,-15],"reward":0,"Value":1.125}}
null	{"999999":{"state":[1,2,3,4,0,5,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,8,0,0,0,0,0,9,10,0,0,11,0,0,0,0,0,0,0,0,0,12,13,0,0,0,14,15,16,0,-16,0,-15],"action":"[Rook,(1,0),(4,7),(5,7)]","state_prime":[1,2,3,4,0,5,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,7,0,0,0,0,0,8,0,0,0,0,0,0,0,9,0,0,0,10,0,0,0,0,11,0,0,0,0,12,13,0,0,0,14,15,16,0,-16,0,-15],"reward":0,"Value":0.25}}
null	{"999999":{"state":[1,2,3,4,0,5,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,7,0,0,0,0,0,8,0,0,0,0,0,0,0,9,0,0,0,10,0,0,0,0,11,0,0,0,0,12,13,0,0,0,14,15,16,0,-16,0,-15],"action":"[Bishop,(1,-1),(6,5),(7,4)]","state_prime":[1,2,3,4,0,5,0,

### Get Transition Matrix for Full Run..

### Compute State_Action Transition Matrix

In [18]:
%%writefile getUniqueStateProbability.py
#!~/usr/bin/python

from __future__ import division
import re
import mrjob
import json,ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict, Counter


class getUniqueStateProbability(MRJob):
    MRJob.SORT_VALUES = False

    def mapper(self, _, line_):
        try:
            x,_line = line_.split("\t")
            line    = ast.literal_eval(_line)
            for cycle in line.keys():
                state   = line[cycle]['state']
                action  = line[cycle]['action']
                state_prime = line[cycle]['state_prime']
                reward  = line[cycle]['reward']
                Value   = line[cycle]['Value']
            
            yield (state, action, state_prime, Value),1
        except:
            yield "failed", None
            
            
    def reducer(self, state_action_prime, count):

        frequency = sum(count)
        state = state_action_prime[0]
        state_action = state_action_prime[:2]
        Value = state_action_prime[3]
        state_prime  = state_action_prime[2]
        prime_freq   = (state_prime, frequency)
        #yield state_action, prime_freq
        yield state, Value

    def steps(self):  
        jobconf2 = {'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                    'stream.num.map.output.key.field':1, 
                    'mapreduce.partition.keycomparator.options': '-k1,k2nr'}  
        return [MRStep(mapper=self.mapper,
                      reducer=self.reducer)]
    
        
if __name__ == '__main__':
    getUniqueStateProbability.run()

Overwriting getUniqueStateProbability.py


In [19]:
STATE_FREQUENCY_OUT = "/data_data/reinforcement_learning/results/STATE_FREQUENCY"

In [20]:
#!rm  /data_data/reinforcement_learning/results/state_value_sarsa_1k.tsv 
#!rm -rf /data_data/reinforcement_learning/results/STATE_FREQUENCY/
#!python getUniqueStateProbability.py -r local /data_data/reinforcement_learning/results/history_sarsa_1k_test --output-dir={STATE_FREQUENCY_OUT} --no-output
#!cat  /data_data/reinforcement_learning/results/STATE_FREQUENCY/part-* >>  /data_data/reinforcement_learning/results/state_value_sarsa_1k.tsv

In [21]:
#!tail -n 5  /data_data/reinforcement_learning/results/state_value_sarsa_1k.tsv

In [22]:
!rm  /data_data/reinforcement_learning/results/state_value_sarsa_2M.tsv 
!rm -rf /data_data/reinforcement_learning/results/STATE_FREQUENCY/
!python getUniqueStateProbability.py -r local /data_data/reinforcement_learning/results/history_sarsa_2M_test.tsv --output-dir={STATE_FREQUENCY_OUT} --no-output
!cat  /data_data/reinforcement_learning/results/STATE_FREQUENCY/part-* >>  /data_data/reinforcement_learning/results/state_value_sarsa_2M.tsv

rm: cannot remove '/data_data/reinforcement_learning/results/state_value_sarsa_2M.tsv': No such file or directory
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/getUniqueStateProbability.ubuntu.20200506.162341.561459
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/STATE_FREQUENCY
Removing temp directory /tmp/getUniqueStateProbability.ubuntu.20200506.162341.561459...


In [23]:
!tail -n 5  /data_data/reinforcement_learning/results/state_value_sarsa_2M.tsv

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]	1.000015
[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]	1.001411
[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]	1.250439
[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]	1.252004
[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]	1.352146


### Compute Average Value for a State

In [24]:
%%writefile getAverageValueForStates.py
#!~/usr/bin/python

from __future__ import division
import re
import mrjob
import json
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict, Counter
import ast

class getAverageValueForStates(MRJob):
    MRJob.SORT_VALUES = False

    def mapper(self, _, line_):
        try:
            #state, values = line_
            state, values = line_.replace("]","]::::").split("::::")
            yield ast.literal_eval(state), float(values[2:])
        except:
            yield None, None

    def reducer(self, state, values):
        temp_arr = [value for value in values]

        try:
            num_values = int(len(temp_arr))
            sum_values = sum(temp_arr)
            avg_value  = round(sum_values/num_values,6)
            yield state, avg_value
        except:
            pass

    def steps(self):  
        jobconf2 = {'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                    'stream.num.map.output.key.field':1, 
                    'mapreduce.partition.keycomparator.options': '-k1,k2nr'}  
        return [MRStep(mapper=self.mapper,
                      reducer=self.reducer)]
    
        
if __name__ == '__main__':
    getAverageValueForStates.run()



Overwriting getAverageValueForStates.py


In [25]:
#!rm  /data_data/reinforcement_learning/results/state_avg_value_sarsa_1k.tsv 
#!rm -rf /data_data/reinforcement_learning/results/STATE_FREQUENCY/
#!python getAverageValueForStates.py -r local /data_data/reinforcement_learning/results/state_value_sarsa_1k.tsv --output-dir={STATE_FREQUENCY_OUT} --no-output
#!cat  /data_data/reinforcement_learning/results/STATE_FREQUENCY/part-* >>  /data_data/reinforcement_learning/results/state_avg_value_sarsa_1k.tsv

In [26]:
#!head -n 10 /data_data/reinforcement_learning/results/state_avg_value_sarsa_1k.tsv

In [27]:
!rm  /data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv 
!rm -rf /data_data/reinforcement_learning/results/STATE_FREQUENCY/
!python getAverageValueForStates.py -r local /data_data/reinforcement_learning/results/state_value_sarsa_2M.tsv --output-dir={STATE_FREQUENCY_OUT} --no-output
!cat  /data_data/reinforcement_learning/results/STATE_FREQUENCY/part-* >>  /data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv

rm: cannot remove '/data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv': No such file or directory
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/getAverageValueForStates.ubuntu.20200506.195936.789930
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/STATE_FREQUENCY
Removing temp directory /tmp/getAverageValueForStates.ubuntu.20200506.195936.789930...


In [28]:
!head -n 10 /data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv

[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]	0.012451
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]	0.435902
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0]	1.398434
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0]	0.206993
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,7,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,10,0,0,0,0,11,0,0,0]	0.003296
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,7,0,0,8,0,0,0,9,0,10,0,0,0,0,0,0,0,11,0,12,0,0,0,0,13,0,0,0]	2.001648
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,7,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,10,0,0,0,0,0,0,0]	0.413986
[0,0,0,0,0,0,0,0,0

In [29]:
!cat /data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv | wc -l

114697411


### POLICY SEARCH

#### Compute Transition Frequencies and Probabilities. Derive Transition matrix

In [30]:
%%writefile getUniqueStateProbability.py
#!~/usr/bin/python

from __future__ import division
import re
import mrjob
import json,ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict, Counter


class getUniqueStateProbability(MRJob):
    MRJob.SORT_VALUES = False

    def mapper(self, _, line_):
        try:
            x,_line = line_.split("\t")
            line    = ast.literal_eval(_line)
            for cycle in line.keys():
                state   = line[cycle]['state']
                action  = line[cycle]['action']
                state_prime = line[cycle]['state_prime']
                reward  = line[cycle]['reward']
                Value   = line[cycle]['Value']
            
            yield (state, action, state_prime),1
        except:
            yield "failed", None
            
            
    def reducer(self, state_action_prime, count):

        frequency = sum(count)
        #state = state_action_prime[0]
        #state_action = state_action_prime[:2]
        #Value = state_action_prime[3]
        #state_prime  = state_action_prime[2]
        #prime_freq   = (state_prime, frequency)
        yield state_action_prime, frequency

    def steps(self):  
        jobconf2 = {'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                    'stream.num.map.output.key.field':1, 
                    'mapreduce.partition.keycomparator.options': '-k1,k2nr'}  
        return [MRStep(mapper=self.mapper,
                      reducer=self.reducer)]
    
        
if __name__ == '__main__':
    getUniqueStateProbability.run()

Overwriting getUniqueStateProbability.py


In [31]:
#!rm  /data_data/reinforcement_learning/results/state_transition_sarsa_1k.tsv 
#!rm -rf /data_data/reinforcement_learning/results/STATE_FREQUENCY/
#!python getUniqueStateProbability.py -r local /data_data/reinforcement_learning/results/history_sarsa_1k_test --output-dir={STATE_FREQUENCY_OUT} --no-output
#!cat  /data_data/reinforcement_learning/results/STATE_FREQUENCY/part-* >>  /data_data/reinforcement_learning/results/state_transition_sarsa_1k.tsv

In [32]:
#!head -n 5  /data_data/reinforcement_learning/results/state_transition_sarsa_1k.tsv

In [33]:
!rm  /data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv 
!rm -rf /data_data/reinforcement_learning/results/STATE_FREQUENCY/
!python getUniqueStateProbability.py -r local /data_data/reinforcement_learning/results/history_sarsa_2M_test.tsv --output-dir={STATE_FREQUENCY_OUT} --no-output
!cat  /data_data/reinforcement_learning/results/STATE_FREQUENCY/part-* >>  /data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv

rm: cannot remove '/data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv': No such file or directory
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/getUniqueStateProbability.ubuntu.20200506.203955.283922
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/STATE_FREQUENCY
Removing temp directory /tmp/getUniqueStateProbability.ubuntu.20200506.203955.283922...


In [34]:
!tail -n 5  /data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv

[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,-15,-14,-13,-12,-11,-10,0,-9,-8,-7,-6,-5,-4,-3,-2,-1]]	4770
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,-15,-14,-13,-12,-11,0,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]	4660
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1

#### Create a compact representation of game states using one-hot encoding

In [35]:
!head -n 5 /data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv

[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]	0.012451
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]	0.435902
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0]	1.398434
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0]	0.206993
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,7,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,10,0,0,0,0,11,0,0,0]	0.003296


In [36]:
!rm /data_data/reinforcement_learning/results/unique_states_2M.tsv
!cat /data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv | awk -F"\t" '{print $1}' >> /data_data/reinforcement_learning/results/unique_states_2M.tsv

rm: cannot remove '/data_data/reinforcement_learning/results/unique_states_2M.tsv': No such file or directory


#### Version I: Get unique states 

In [37]:
#import ast
#from collections import defaultdict

#unique_states_file = "/data_data/reinforcement_learning/results/unique_states_500K.tsv"

#unique_states_dict = defaultdict()

#state_index = 0

#with open(unique_states_file,'r') as all_unique_states:
    
#    unique_states = all_unique_states.readlines()
    
#    for unique_state in unique_states:
        
#        unique_states_dict[unique_state.strip()] = state_index
        
#        state_index += 1


    
    

In [38]:
#len(unique_states_dict.keys())

#### Version II: Get unique states

In [39]:
!tail -n 5  /data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv

[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,-15,-14,-13,-12,-11,-10,0,-9,-8,-7,-6,-5,-4,-3,-2,-1]]	4770
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,-15,-14,-13,-12,-11,0,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]	4660
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1

In [40]:
%%writefile getUniqueStateOrderedByCounts.py
#!~/usr/bin/python

from __future__ import division
import re
import mrjob
import json,ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict, Counter


class getUniqueStateOrderedByCounts(MRJob):
    MRJob.SORT_VALUES = False

    def mapper(self, _, line_):
        try:
            x,_line = line_.split("\t")
            line    = ast.literal_eval(_line)
            for cycle in line.keys():
                state   = line[cycle]['state']
                action  = line[cycle]['action']
                state_prime = line[cycle]['state_prime']
                reward  = line[cycle]['reward']
                Value   = line[cycle]['Value']
            
            yield state,1
            yield state_prime, 1
        except:
            yield "failed", None
            
            
    def reducer(self, state, count):
        frequency = sum(count)
        yield state, frequency

    def steps(self):  
        jobconf2 = {'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                    'stream.num.map.output.key.field':1, 
                    'mapreduce.partition.keycomparator.options': '-k1,k2n'}  
        return [MRStep(mapper=self.mapper,
                      reducer=self.reducer)]
    
        
if __name__ == '__main__':
    getUniqueStateOrderedByCounts.run()

Overwriting getUniqueStateOrderedByCounts.py


In [41]:
!rm  /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K.tsv
!rm  /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K_.tsv
!rm /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K__.tsv

!rm -rf /data_data/reinforcement_learning/results/STATE_COUNTS/

!python getUniqueStateOrderedByCounts.py -r local /data_data/reinforcement_learning/results/history_sarsa_1k_test --output-dir=/data_data/reinforcement_learning/results/STATE_COUNTS/ --no-output

!cat  /data_data/reinforcement_learning/results/STATE_COUNTS/part-* >>  /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K__.tsv

!tac /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K__.tsv >> /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K_.tsv

!nawk '$1=(FNR FS $1)' /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K_.tsv >> /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K.tsv 

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/getUniqueStateOrderedByCounts.ubuntu.20200507.000838.778440
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/STATE_COUNTS/
Removing temp directory /tmp/getUniqueStateOrderedByCounts.ubuntu.20200507.000838.778440...


In [42]:
!cat /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K.tsv | wc -l

499


In [43]:
!head  /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_1K.tsv

1 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1] 25
2 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,0,-1] 4
3 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,0,0,0,-15,0,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1] 4
4 [1,2,3,4,5,6,7,8,9,10,11,0,12,13,14,15,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,0,-1] 4
5 [1,2,3,4,5,6,7,8,9,10,11,0,12,13,14,15,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,-15,-14,-13,-12,-11,0,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1] 4
6 [1,2,3,4,5,6,7,8,9,10,11,0,12,13,14,0,0,0,15,0,0,0,16,0,0,0,0,0,-16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,0,-1] 4
7 [1,2,3,4,5,6,

In [44]:
!rm /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M.tsv

!rm -rf /data_data/reinforcement_learning/results/STATE_COUNTS/

!python getUniqueStateOrderedByCounts.py -r local /data_data/reinforcement_learning/results/history_sarsa_2M_test.tsv --output-dir=/data_data/reinforcement_learning/results/STATE_COUNTS/ --no-output

!cat  /data_data/reinforcement_learning/results/STATE_COUNTS/part-* >>  /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M__.tsv

!tac  /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M__.tsv >> /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M_.tsv
!rm /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M__.tsv

!nawk '$1=(FNR FS $1)' /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M_.tsv >> /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M.tsv 
!rm /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M_.tsv

rm: cannot remove '/data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M.tsv': No such file or directory
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/getUniqueStateOrderedByCounts.ubuntu.20200507.000858.671330
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/STATE_COUNTS/
Removing temp directory /tmp/getUniqueStateOrderedByCounts.ubuntu.20200507.000858.671330...


In [45]:
!head /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M.tsv 

1 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1] 6000000
2 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,0,-15,0,0,-14,0,-13,-12,-11,-10,-9,-8,-7,0,-6,-5,-4,-3,-2,-1] 4
3 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,0,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,0,-6,-5,-4,-3,-2,-1] 8
4 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,0,-6,-5,-4,-3,-2,-1] 4
5 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,0,-6,-5,-4,-3,-2,-1] 4
6 [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,0,0,0,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,0,-1] 4
7 [1,2,3,4

In [46]:
!head -n 2000 /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M.tsv  >> /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2K.tsv

In [47]:
!cat /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M.tsv | wc -l

114697411


In [48]:
%%writefile createUniqueStatesDict.py
#!~/usr/bin/python

from __future__ import division
import re
import mrjob
import json,ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict, Counter


class createUniqueStatesDict(MRJob):
    MRJob.SORT_VALUES = False

    def mapper(self, _, line_):
        try:
            unique_id = int(line_.split(" [")[0])
            state = str("[" + line_.split(" [")[1].split("] ")[0] + "]")
            yield state, unique_id
        except:
            pass
            
            
    def reducer(self, unique_state, unique_ids):
        
        unique_states_dict = defaultdict()
        
        for unique_id in unique_ids:
            yield "{'"+str(unique_state)+ "':" + str(unique_id)+"}", "\t"


        

    def steps(self):  
        jobconf2 = {'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                    'stream.num.map.output.key.field':1, 
                    'mapreduce.partition.keycomparator.options': '-k1,k2n'}  
        return [MRStep(mapper=self.mapper,
                      reducer=self.reducer)]
    
        
if __name__ == '__main__':
    createUniqueStatesDict.run()

Overwriting createUniqueStatesDict.py


In [49]:
!rm /data_data/reinforcement_learning/results/unique_states_2K.tsv

!rm -rf /data_data/reinforcement_learning/results/UNIQUE_STATES/

!python createUniqueStatesDict.py -r local /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2K.tsv --output-dir=/data_data/reinforcement_learning/results/UNIQUE_STATES/ --no-output

!sed 's/"//g'  /data_data/reinforcement_learning/results/UNIQUE_STATES/part-* | awk -F "\t" '{print $1}'  >>  /data_data/reinforcement_learning/results/unique_states_2K.tsv



No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/createUniqueStatesDict.ubuntu.20200507.030356.532042
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/UNIQUE_STATES/
Removing temp directory /tmp/createUniqueStatesDict.ubuntu.20200507.030356.532042...


In [50]:
!head /data_data/reinforcement_learning/results/unique_states_2K.tsv 

{'[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, -16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]':2000}
{'[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, -16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]':2000}
{'[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, -16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, -14, -13, -12, -11, -10, -9, -8, -7, 0, -6, -5, -4, -3, -2, -1]':1999}
{'[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, -16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, -14, -13, -12, -11, -10, -9, -8, -7, 0, -6, -5, -4, -3, -2, -1]':1999}
{'[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15, 0, 0, 0, 0, 0, 

In [51]:
!rm /data_data/reinforcement_learning/results/unique_states_2M.tsv

!rm -rf /data_data/reinforcement_learning/results/UNIQUE_STATES/

!python createUniqueStatesDict.py -r local /data_data/reinforcement_learning/results/unique_states_ordered_by_counts_2M.tsv --output-dir=/data_data/reinforcement_learning/results/UNIQUE_STATES/ --no-output

!sed 's/"//g'  /data_data/reinforcement_learning/results/UNIQUE_STATES/part-* | awk -F "\t" '{print $1}'  >>  /data_data/reinforcement_learning/results/unique_states_2M.tsv



No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/createUniqueStatesDict.ubuntu.20200507.030406.209119
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/UNIQUE_STATES/
Removing temp directory /tmp/createUniqueStatesDict.ubuntu.20200507.030406.209119...


In [52]:
import ast
from collections import defaultdict

unique_states_file = "/data_data/reinforcement_learning/results/unique_states_2M.tsv"

unique_states_dict = defaultdict()

state_index = 0

with open(unique_states_file,'r') as all_unique_states:
    
    transitions = all_unique_states.readlines()
    
    for transition in transitions:
        
        unique_states_dict.update(ast.literal_eval(transition))


    
    

In [53]:
len(unique_states_dict.keys())

114697411

In [54]:
len(unique_states_dict.items())

114697411

In [55]:
list(unique_states_dict.keys())[:5]

['[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]',
 '[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]',
 '[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0]',
 '[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0]',
 '[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,7,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,10,0,0,0,0,11,0,0,0]']

#### One hot encode Transition Matrix

In [56]:
!tail -n 5  /data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv

[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,-15,-14,-13,-12,-11,-10,0,-9,-8,-7,-6,-5,-4,-3,-2,-1]]	4770
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,-15,-14,-13,-12,-11,0,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]	4660
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,6),(2,7)]",[1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1

##### revisit this code block -- to extract total number of transitions earlier....

In [57]:
!cat /data_data/reinforcement_learning/results/state_transition_one_hot_2M.tsv | awk -F"\t" '{print $2}' >> /data_data/reinforcement_learning/results/state_transition_one_hot_sum_frequencies_2M.tsv

cat: /data_data/reinforcement_learning/results/state_transition_one_hot_2M.tsv: No such file or directory


In [58]:
# Need to calculate this prior
total_transitions = 12100292

In [59]:
!ls -alrth "/data_data/reinforcement_learning/results/state_transition_one_hot_sum_frequencies_2M.tsv"

-rw-rw-r-- 1 ubuntu ubuntu 0 May  6 21:52 /data_data/reinforcement_learning/results/state_transition_one_hot_sum_frequencies_2M.tsv


In [60]:
transition_frequencies_file = "/data_data/reinforcement_learning/results/state_transition_one_hot_sum_frequencies_2M.tsv"

total_transitions = 0

with open(transition_frequencies_file,'r') as trans_freq_file:
    transition_frequencies = trans_freq_file.readlines()
    for transition_frequency in transition_frequencies:
        total_transitions += round(float(transition_frequency.strip()))

print("total_transitions:\t{}".format(total_transitions))

total_transitions:	0


In [61]:
total_transitions

0

##### Calculate transition frequencies and use one-hot encoding to record states

##### First calculate frequency of (s,a) pairs

In [62]:
!head -n 5 /data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv

[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0],"[Bishop,(1,1),(4,0),(5,1)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,3,0,4,0,0,0,5,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0],"[Rook,(1,0),(3,0),(4,0)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,3,0,0,0,0,4,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0],"[Rook,(1,0),(3,7),(4,7)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,3,4,5,0,0,0,0,0,0,0,0,0,6,0,0,0,0,7,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,

In [63]:
%%writefile getStateActionFrequencies.py
#!~/usr/bin/python

from __future__ import division
import re
import mrjob
import json,ast
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import defaultdict, Counter


class getStateActionFrequencies(MRJob):
    MRJob.SORT_VALUES = False

    def mapper(self, _, state_action_prime_frequency):
        try:
            state_action = ast.literal_eval((state_action_prime_frequency.split("\t")[0]))[:2]
            
            yield state_action,int(1)
        except:
            pass
            
            
    def reducer(self, state_action, counts):
        
        yield state_action, sum(counts)

    def steps(self):  
        jobconf2 = {'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                    'stream.num.map.output.key.field':1, 
                    'mapreduce.partition.keycomparator.options': '-k1,k2nr'}  
        return [MRStep(mapper=self.mapper,
                      reducer=self.reducer)]
    
        
if __name__ == '__main__':
    getStateActionFrequencies.run()

Overwriting getStateActionFrequencies.py


In [64]:
from collections import defaultdict
state_action_transitions_dict = defaultdict()

In [65]:
state_transitions_file_verbose = "/data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv"

state_action_transitions_dict = defaultdict()

!rm -rf /data_data/reinforcement_learning/results/STATE_FREQUENCY/

!rm /data_data/reinforcement_learning/results/state_action_frequencies_2M.tsv

!python getStateActionFrequencies.py -r local /data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv --output-dir=/data_data/reinforcement_learning/results/STATE_FREQUENCY/ 

!cat /data_data/reinforcement_learning/results/STATE_FREQUENCY/part-* >> /data_data/reinforcement_learning/results/state_action_frequencies_2M.tsv

rm: cannot remove '/data_data/reinforcement_learning/results/state_action_frequencies_2M.tsv': No such file or directory
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/getStateActionFrequencies.ubuntu.20200507.035257.158396
Running step 1 of 1...
job output is in /data_data/reinforcement_learning/results/STATE_FREQUENCY/
Removing temp directory /tmp/getStateActionFrequencies.ubuntu.20200507.035257.158396...


In [66]:
!tail -n 10 /data_data/reinforcement_learning/results/state_action_frequencies_2M.tsv

[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,0),(1,5),(2,5)]"]	26
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,0),(1,6),(2,6)]"]	26
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,0),(1,7),(2,7)]"]	26
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,0),(2,1)]"]	26
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"[Pawn,(1,1),(1,1),(2,2)]"]	26
[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16

In [67]:
from collections import defaultdict

import ast

sa_transition_probs = defaultdict()

sa_freqs = "/data_data/reinforcement_learning/results/state_action_frequencies_2M.tsv"

with open(sa_freqs,'r') as sa_freqs_file:
    sa_frequencies = sa_freqs_file.readlines()
    for sa_frequency_ in sa_frequencies:
        #state, action, frequency = ast.literal_eval(sa_frequency_.strip().replace("\"","'").split("\t"))
        state_action, frequency = sa_frequency_.strip().replace("\"","'").split("\t")

        sa_transition_probs[state_action] = int(frequency)

#sa_transition_probs.items()

In [68]:
len(sa_transition_probs.items())

116271973

In [69]:
!head "/data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv"

[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0],"[Bishop,(1,1),(4,0),(5,1)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,3,0,4,0,0,0,5,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0],"[Rook,(1,0),(3,0),(4,0)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,3,0,0,0,0,4,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0],"[Rook,(1,0),(3,7),(4,7)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,3,4,5,0,0,0,0,0,0,0,0,0,6,0,0,0,0,7,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,

#### one-hot encode average values for state

In [70]:
! head -n 4 /data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv

[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]	0.012451
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]	0.435902
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0]	1.398434
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0]	0.206993


In [71]:
avg_state_values_file  = "/data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv"

state_values_dict = defaultdict()

with open(avg_state_values_file,'r') as avg_state_val_file:
    
    avg_state_values = avg_state_val_file.readlines()
    
    for avg_state_val in avg_state_values:
        
        try:

            state_, avg_val_ = avg_state_val.split("\t") 

            state = unique_states_dict[state_]

            state_values_dict[state] = float(avg_val_)
        
        except:
            pass


In [72]:
list(state_values_dict.items())[:5]

[(114697411, 0.012451),
 (114697410, 0.435902),
 (114697409, 1.398434),
 (114697408, 0.206993),
 (114697407, 0.003296)]

##### create one-hoe encoded transition matrix

In [73]:
!head -n 5 "/data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv"

[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0],"[Bishop,(1,1),(4,0),(5,1)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,3,0,4,0,0,0,5,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0],"[Rook,(1,0),(3,0),(4,0)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,3,0,0,0,0,4,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0],"[Rook,(1,0),(3,7),(4,7)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,3,4,5,0,0,0,0,0,0,0,0,0,6,0,0,0,0,7,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,

##### Option I <= fix issue of groupby (s,a)

In [75]:
#sas_trans_dict[2]

In [ ]:
import json

damping_factor = 5

default_freq   = 30

sas_transitions_dict = defaultdict()


!rm "/data_data/reinforcement_learning/results/state_transition_one_hot_2M.tsv"

state_transitions_file_verbose = "/data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv"

state_transitions_file_1_hot   = "/data_data/reinforcement_learning/results/state_transition_one_hot_2M.tsv"

!touch "/data_data/reinforcement_learning/results/state_transition_one_hot_2M.tsv"

new_state_starting_index = total_transitions + 10000000000

with open(state_transitions_file_verbose,'r') as verbose_transitions_file:
    
    verbose_transitions = verbose_transitions_file.readlines()
    
    #with open(state_transitions_file_1_hot, 'a') as one_hot_transitions_file:
        
    for verbose_transition in verbose_transitions:
            
            transitions_, frequency_ = verbose_transition.split("\t")
            
            transitions = ast.literal_eval(transitions_)
            
            try:
                state   = unique_states_dict[str(transitions[0])]
            except:
                unique_states_dict[str(transitions[0])] = new_state_starting_index
                new_state_starting_index += 1
                
            try:
                state_prime = unique_states_dict[str(transitions[2])]
            except:
                unique_states_dict[str(transitions[2])] = new_state_starting_index
                new_state_starting_index += 1
            try:
                transition_frequency = round(int(frequency_)/int(sa_transition_probs[str(ast.literal_eval(transitions_)[:2])]+ damping_factor), 8)
            except:
                transition_frequency = round(int(frequency_)/20)
            
            action = transitions[1]
            
            try:
                value_prime  = state_values_dict[state]
                
            except:
                value_prime  = 0
                
            try:
                score_prime  = float(value_prime*transition_frequency) 
            except:
                score_prime  = 0
                
            try:
                
                transition = {action: score_prime}
                
                if not state in sas_transitions_dict.keys():
                    
                    sas_transitions_dict[state] = {}

                sas_transitions_dict[state].update(transition)


            except:
                print("state:\t{}\taction:\t{}\t state_prime:\{}\t FAILED".format(state, action, state_prime))
                    
with open(state_transitions_file_1_hot, 'a') as one_hot_transitions_file:
        one_hot_transitions_file.write(json.dumps(sas_transitions_dict))
            
        
    

rm: cannot remove '/data_data/reinforcement_learning/results/state_transition_one_hot_2M.tsv': No such file or directory


In [ ]:
print("NUM_MOVES\tNUM_STATES")
for i in range(0,50):
    num_states = len(list([x for x in sas_transitions_dict.values() if len(x.items()) == i]))
    print("{}\t\t{}".format(i,num_states))

In [ ]:
list([x for x in sas_transitions_dict.values() if len(x.items()) > 20])[:5]

In [ ]:
sas_transitions_dict[4554341]

In [ ]:
!tail -n 50 /data_data/reinforcement_learning/results/state_transition_one_hot_2M.tsv

In [ ]:
!tail -n 5000 /data_data/reinforcement_learning/results/state_transition_one_hot_2M.tsv

#### Use Spark -- Evaluate Memory Based operations

In [9]:
APP_NAME = "Policy Evaluation - Chessy"

try:
    sc.stop()
    spark.stop()
except:
    pass

from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

    
sc = SparkContext(conf = SparkConf() 
                  .set("spark.driver.maxResultSize", "40g") 
                  .set("spark.sql.execution.arrow.enabled", "true") 
                  .set('spark.sql.broadcastTimeout', 1000) 
                  .set('spark.local.dir', '/data_data/session_length/spark_tmp/') 
                  .set('spark.driver.memory', '60G') 
                  .set("spark.executor.instances", "20") 
                  .set("spark.executor.cores", 16) 
                  .set("spark.executor.memory", "10G")).getOrCreate()
spark = SparkSession(sc)
spark.sparkContext.setLogLevel("ERROR")

In [123]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DateType, DoubleType, IntegerType, ArrayType
from pyspark.sql.functions import count, mean, stddev_pop, min, max, lit, round, bround, pow, col, corr, lower, upper, avg, stddev, abs, log
from pyspark.sql.functions import lit, trim, rtrim, rpad, trim, coalesce
from pyspark.sql.functions import current_date, current_timestamp, date_add, date_sub, months_between, to_date
from pyspark.sql.functions import udf, col, sum, from_json
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank, rank, expr, split, regexp_replace


from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from datetime import datetime

In [124]:
sc._conf.getAll()

[('spark.local.dir', '/data_data/session_length/spark_tmp/'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.executor.instances', '20'),
 ('spark.driver.port', '40991'),
 ('spark.driver.host', '192.168.1.101'),
 ('spark.driver.maxResultSize', '40g'),
 ('spark.executor.memory', '10G'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.cores', '16'),
 ('spark.master', 'local[*]'),
 ('spark.driver.memory', '60G'),
 ('spark.submit.deployMode', 'client'),
 ('spark.sql.broadcastTimeout', '1000'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1588906700495')]

In [ ]:
!head -n 5 "/data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv"

In [47]:
!head -n 5 "/data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv"

[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0],"[Bishop,(1,1),(4,0),(5,1)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,3,0,4,0,0,0,5,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0],"[Rook,(1,0),(3,0),(4,0)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,3,0,0,0,0,4,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0],"[Rook,(1,0),(3,7),(4,7)]",[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,3,4,5,0,0,0,0,0,0,0,0,0,6,0,0,0,0,7,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0]]	2
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,

In [48]:
#!head -n 50 "/data_data/reinforcement_learning/results/state_transition_sarsa_2M.tsv" >> test_import.tsv

In [49]:
SARF_df = None

In [87]:


def import_sarsa_data():
    global SARF_df
    print("====================================================================================================================")
    print("====================================state_avg_value_sarsa_2M.tsv====================================================")
    print("==================================================================================================================\n\n")
    DRLSchema = StructType([\
        StructField('SAS',  StringType(), True),\
        StructField('frequency',  IntegerType(), True)])
    
    SARF = spark.read.format('csv').schema(DRLSchema).option("sep","\t").load('test_import.tsv')
    SARF.show(5,False)

    
    SARF_df = SARF.withColumn('stateprior', split(SARF['SAS'], '\]')).withColumn('state_prior', regexp_replace(col('stateprior')[0], "\[\[", "")).withColumn('action', regexp_replace(col('stateprior')[1], "\,\"\[", "")).withColumn('state_prime', regexp_replace(col('stateprior')[2], "\",\[", "")).drop('stateprior').drop('SAS')
    SARF_df.show(5,False)
    


In [88]:
import_sarsa_data()

====================================state_avg_value_sarsa_2M.tsv====================================================


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|SAS                                                                                                                                                                                                                                                                                                  |frequency|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [90]:
!head -n 5 /data_data/reinforcement_learning/results/state_avg_value_sarsa_2M.tsv

[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,0,4,0,5,0,0,0,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0]	0.012451
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,6,0,7,0,0,0,0,0,0,0,0,8,9,0,10,0,0,0,0]	0.435902
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,3,4,5,0,6,0,0,0,0,0,0,0,7,0,0,0,0,8,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0]	1.398434
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,0,7,0,0,0,0,8,0,0,0,9,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0]	0.206993
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,0,4,5,0,6,0,0,0,0,7,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,10,0,0,0,0,11,0,0,0]	0.003296


In [93]:
!head -n 20 /data_data/reinforcement_learning/results/history_sarsa_2M_test.tsv > sample_sarsa_out

In [116]:
!head json_import_test.json

{"game": {"0":{"state":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1],"action":"[Knight,(2,-1),(0,1),(2,0)]","state_prime":[1,0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,0,-6,-5,-4,-3,-2,-1],"reward":0,"Value":0.637459}},
 {"0":{"state":[1,0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-16,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,0,-6,-5,-4,-3,-2,-1],"action":"[Knight,(1,2),(2,0),(3,2)]","state_prime":[1,0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0,0,0,0,0,0,0,0,0,0,16,-16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,0,-6,-5,-4,-3,-2,-1],"reward":0,"Value":1.274919}},
 {"0":{"state":[1,0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0,0,0,0,0,0,0,0,0,0,16,-16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-15,-14,-13,-12,-11,-10,-9,-8,-7,0

In [137]:
SARSA = None

def import_sarsa_data():
    global SARSA
    print("====================================================================================================================")
    print("==================================== history_sarsa_2M_test.tsv​ ====================================================")
    print("====================================================================================================================\n\n")
    DRLSchema = StructType([\
        StructField('Null',  StringType(), True),\
        StructField('SARSA',  StringType(), True)])
    
    SARSA = spark.read.format('csv').schema(DRLSchema).option("sep","\t").load('sample_sarsa_json')
    SARSA.show(5,False)


In [138]:
import_sarsa_data()

==================================== history_sarsa_2M_test.tsv​ ====================================================


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Null                                                                                                                                                                                                                                                                                                                                                                                                                             |SARSA|
+----------------------------

In [139]:
schema = StructType(
    [
        StructField('state_prior', StringType(), True),
        StructField('action',      StringType(), True),
        StructField('state_prime', StringType(), True),
        StructField('reward',      StringType(), True),
        StructField('value',       StringType(), True)
    ]
)

SARSA.withColumn("new_SARSA", from_json("SARSA", schema))\
    .show(5, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+---------+
|Null                                                                                                                                                                                                                                                                                                                                                                                                                             |SARSA|new_SARSA|
+-------------------------------------------------------------------------------------------------------------------------------

In [ ]:
users_df = users_df.withColumnRenamed('userid','user_id')
u = users_df.alias('u')
s = sessions_DF.alias('s')
join_condition = [ (u.user_id == s.userid) ]
sessionsDF = s.join(u, join_condition, 'inner').drop('user_id')
sessionsDF.show(20,False)